In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import requests
import pandas as pd
import re

import url


In [ ]:

url = "https://newsapi.org/v2/everything?q=tesla&from=2025-03-15&sortBy=publishedAt&apiKey=3b1dd03b6308473cbec1edab9ac53180"
response = requests.get(url)

# Get the JSON data from the response
data = response.json()

# Now you can use 'get' on the JSON data
articles = data.get("articles", [])
data = []
for article in articles:
    data.append({
        "title": article["title"],
        "description": article["description"]
    })

df = pd.DataFrame(data)

csv = df.to_csv('data.csv', index=False)

clean text data


In [ ]:
def clean(text):
     text = re.sub(r"http\S+", "", str(text))
     text = re.sub(r"[^a-zA-Z\s\u4e00-\u9fff]", "", str(text))
     return text.lower()

df["clean_title"] = df["title"].apply(clean)


Text Blob

In [ ]:
from textblob import TextBlob

def get_sentiment(text):
    return TextBlob(text).sentiment.polarity

df["sentiment"] = df["clean_title"].apply(get_sentiment)

**add pipeline**

In [ ]:
from transformers import pipeline

sentiment_model = pipeline("sentiment-analysis")

df["sentiment"] = df["clean_title"].apply(lambda x: sentiment_model(x)[0] if isinstance(x, str) and x.strip() != '' else {"label": "NEUTRAL", "score": 0})
df["label"] = df["sentiment"].apply(lambda x: x['label'])
df["score"] = df["sentiment"].apply(lambda x: x['score'])

print(df.head(10))

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


                                               title  \
0              Tesla Stock Plummets 50%: More To Go?   
1  La grande mise à jour Tesla de printemps arriv...   
2  Редкоземельные металлы Китая: ограничения пост...   
3  Le Tesla Cybertruck devient plus abordable, ce...   
4  US-China Trade War: Beijing Halts Rare Earth E...   
5                Sverige satsar på elmopeder i Ghana   
6  Warren Buffett Bought This Electric Vehicle (E...   
7  El bromance ya advertía grietas; Casa Blanca a...   
8  Elon Musk Could Be Democrats' Secret Weapon in...   
9  Tesla Stock Rises. Earnings Expectations Are ‘...   

                                         description  \
0  Tesla stock appears relatively expensive, maki...   
1  Tesla profite de sa "Sping Release" pour propo...   
2  Китай, контролирующий значительную долю рынка ...   
3  La version d’entrée de gamme du pick-up de Tes...   
4  China halts rare earth exports in tariff war r...   
5  För att kompensera våra klimatutsläpp \n\n\n